In [22]:
import os, sys
import re
import pandas as pd
import numpy as np

dir_path = "Files2"
ls_dir = os.listdir(dir_path)
year = 0;

# Finding Directory or year for which csv are present
for file in ls_dir:
    regexp = re.compile(r'.txt')
    print(file)
    if not(regexp.search(file)):
        year = file
    
# Setting Directory for the year    
if not(year == 0):
    print(year)
    dir_path += "/" + str(year)
else:
    print("No Files found! Ending Program")
    sys.exit()
    
#Looping over all the csv to load and process the data
ls_dir = os.listdir(dir_path)

i=1;

for file in ls_dir:
#     only if file is csv
    regexp = re.compile(r'.csv')
    if(regexp.search(file)):
        filePath = dir_path + "/" + file
    #   Reading File data with pandas
        fileData = pd.read_csv(filePath,header = 0)

        
#       adding columns for year, month and day of month
        fileData = split_date(fileData)
#       adding columns for hour, minutes and seconds
        fileData = split_time(fileData)
#       adding column for true extension
        fileData = true_extention(fileData)
#       replace empty "size" with 0 if code equals 304
        fileData = clean_size(fileData) 
#       removinf code with value 0
        fileData = remove_code(fileData)
#     replacing all other NaNs with 99999 
        fileData = handling_leftovers(fileData)
        print(fileData.shape)
#      processed data  
        processed_csvs(fileData, filePath,i)
        i=i+1;
print("END OF PROGRAM")


2003
2003
(0, 23)
(0, 23)
(43458, 23)
(261289, 23)
(291590, 23)
(114672, 23)
(373249, 23)
(238765, 23)
(63860, 23)
(119840, 23)
(10330, 23)
(166346, 23)
END OF PROGRAM


In [10]:
def split_date(fileData):
    splitter = fileData['date'].apply(lambda x: x.split('-'))
    fileData['year'] = splitter.apply(lambda x: x[0])
    fileData['month'] = splitter.apply(lambda x: x[1])
    fileData['dayOfMonth'] = splitter.apply(lambda x: x[2])
    return fileData
    

In [11]:
def clean_size(fileData):
    if (any(fileData['code'] == 304)):
        fileData['size'].fillna(0, inplace=True)
    return fileData

In [12]:
def split_time(fileData):
    splitter = fileData['time'].apply(lambda x: x.split(':'))
    fileData['h'] = splitter.apply(lambda x: x[0])
    fileData['m'] = splitter.apply(lambda x: x[1])
    fileData['s'] = splitter.apply(lambda x: x[2])
    return fileData

In [13]:
def create_directory(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

In [14]:
def remove_code(fileData):
    fileData = fileData[(fileData['code'] != 0)]
    return fileData

In [21]:
def processed_csvs(fileData, filePath,i):
    fp = re.split('/', filePath, flags=re.IGNORECASE)
    new_file_path = fp[0] + "/" + fp[1]  + "/"
    create_directory(new_file_path + "Cleaned_Files")
    new_file_path += "Cleaned_Files/Cleaned_" + fp[2] 
    fileData.to_csv(new_file_path, sep=',')
#     if(i==1):
#         fileData.to_csv(new_file_path, sep=',')
#     else:
#         with open(new_file_path, 'a') as f:
#             fileData.to_csv(f, header=False)
#     i=i+1


In [16]:
def true_extention(fileData):
    splitter = fileData['extention'].apply(lambda x: x.split('.'))
    try:
        fileData['file_name'] = splitter.apply(lambda x: x[0])
        fileData['true_extention'] = splitter.apply(lambda x: x[1])
#         no true_extention created
    except:
        pass
    return fileData

In [17]:
def handling_leftovers(fileData):
    fileData['file_name'].replace(r'$^', np.nan, regex=True, inplace = True)
    fileData.fillna(999999, inplace=True)
#     print("_______________________________")
#     print(fileData['file_name'])
    
    return fileData